# Day 9: Smoke Basin

[https://adventofcode.com/2021/day/9](https://adventofcode.com/2021/day/9)

## Description

### Part One

These caves seem to be [lava tubes](https://en.wikipedia.org/wiki/Lava_tube). Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly <span title="This was originally going to be a puzzle about watersheds, but we're already under water.">settles like rain</span>.

If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).

Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678
    

Each number corresponds to the height of a particular location, where `9` is the highest and `0` is the lowest a location can be.

Your first goal is to find the _low points_ - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)

In the above example, there are _four_ low points, all highlighted: two are in the first row (a `1` and a `0`), one is in the third row (a `5`), and one is in the bottom row (also a `5`). All other locations on the heightmap have some lower adjacent location, and so are not low points.

The _risk level_ of a low point is _1 plus its height_. In the above example, the risk levels of the low points are `2`, `1`, `6`, and `6`. The sum of the risk levels of all low points in the heightmap is therefore _`15`_.

Find all of the low points on your heightmap. _What is the sum of the risk levels of all low points on your heightmap?_

In [31]:
import numpy as np

input = np.array([[int(c) for c in line.strip()] for line in open('./input/day9.txt').readlines()], dtype=int)
input = np.pad(input, 1, 'constant', constant_values=9)

def findLowestPoints(array):
    lowest_points = []
    for y in range(1, input.shape[0] - 1):
        for x in range(1, input.shape[1] - 1):
            lowest = True
            for yy in range(y - 1, y + 2):
                for xx in range(x - 1, x + 2):
                    if input[y, x] > input[yy, xx]:
                        lowest = False
            
            if lowest:
                lowest_points.append((y, x))
    
    return lowest_points


# find lowest points
lowest_points = findLowestPoints(input)
print('Part #1:', sum(input[c] for c in lowest_points) + len(lowest_points))


Part #1: 444


### Part Two

Next, you need to find the largest basins so you know what areas are most important to avoid.

A _basin_ is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height `9` do not count as being in any basin, and all other locations will always be part of exactly one basin.

The _size_ of a basin is the number of locations within the basin, including the low point. The example above has four basins.

The top-left basin, size `3`:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678
    

The top-right basin, size `9`:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678
    

The middle basin, size `14`:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678
    

The bottom-right basin, size `9`:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678
    

Find the three largest basins and multiply their sizes together. In the above example, this is `9 * 14 * 9 = 1134`.

_What do you get if you multiply together the sizes of the three largest basins?_

In [32]:
def findBasin(array, start, points=None):
    points = points if points else []
    y, x = start

    # reached a hill
    if array[start] == 9 or start in points:
        return

    # add point to basin
    points.append(start)

    # go right
    if x + 1 < array.shape[1]:
        findBasin(array, (y, x+1), points)

    # go down
    if y + 1 < array.shape[0]:
        findBasin(array, (y+1, x), points)

    # go left
    if x - 1 > 0:
        findBasin(array, (y, x-1), points)

    # go up
    if y - 1 > 0:
        findBasin(array, (y-1, x), points)

    return points

def findBasins(array):
    return [findBasin(array, point) for point in findLowestPoints(array)]

# find three largest basins
largest_basins = sorted([len(basin) for basin in findBasins(input)], reverse=True)
print('Part #2:', largest_basins[0] * largest_basins[1] * largest_basins[2])

Part #2: 1168440
